# Python语言与经济大数据分析

# 第5_3讲  网络爬虫实战案例

In [ ]:
# 郭峰（上海财经大学投资系副教授）
#Email：guo.feng@mail.sufe.edu.cn
#2018-07-16
#本讲目录
#5.3.1 政府工作报告-上海现代服务业联合会
#5.3.2 政府工作报告-礼拜五秘书网
#5.3.3 政府工作报告-知县网
#5.3.4 国家哲学社会科学学术期刊
#5.3.5 中国知网
#5.3.6 国家哲学社会科学学术期刊

In [ ]:
#【实际应用案例】：政府工作报告的爬取,来自上海现代服务业联合会
from urllib import request, parse
from urllib.request import urlopen
from urllib.error import HTTPError,URLError
from bs4 import BeautifulSoup 
import time  
import re
head={}  #字典结构
head['User-Agent']='Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36'

for i in range(37000,37015): #最高37015
    url='http://www.ssfcn.com/detailed_gh.asp?id='+str(i)+'&sid='+'\d+'
#   url='http://www.ssfcn.com/detailed_gh.asp?id=36816&sid=2152'模仿此公式计算而来,搞半天才发现“6&sid=2152”是没有意义的。
    print(url)
    try:
        req=request.Request(url,headers=head)
        resp=request.urlopen(url)
        html=resp.read()
        string=html.decode(encoding='utf-8',errors='ignore')  #格式：str.decode(encoding='UTF-8',errors='strict')       
#        print(string)
        title=re.findall('<h1 style=" font-size:14px; color:#990000; padding-top:5px">(.*?)</h1>',string)
        print(title)
        date=re.findall('<P align=center>(.*?)</P>',string,re.S)
#        <P align=center>太原市市长&nbsp; 李荣怀</P>
#        print(date)
        text=re.findall('<P>(.*?)</P>',string,re.S)
#        print(text)
        contents=title+date+text                   #这是个列表，每个元素是一串文字，而不是一个
#        for content in contents:
#            print(content+'\n')
        path='D:/python/baogao_county/'
        if title==[]:                 #个别文档标题为[]，所以先判断一下。
            break
        bgtitle=""
        if  '政府工作报告' in title[0]:    #政府工作报告输出全标题，其他不输出，否则老出错！！！
            bgtitle=title[0]
        f=open(path+'_'+str(i)+'_'+bgtitle+'.txt','w+',encoding='utf-8')  #不加这个encoding，会出错，真是累死人
        for content in contents:
            f.write(content+'\n')
        f.close()
        time.sleep(1)           #爬取间隔数据，逐渐调试,最后发现2也还是会被封。
    except HTTPError as e:
        pass  

In [ ]:
t='_18994_2006年自贡市富顺县政府工作报告.txt'
a=t.find('年')
print(t[1:a-5])

In [ ]:
#【实例操作】上海现代服务业联合会删除无用东西，然后放入另一个文件夹
import os    
import sys
import pickle
import re
import codecs
import string
import shutil
path='D:/python/baogao_county/lianhehui/'
pathn='D:/python/baogao_county/lianhehuinew/'
files= os.listdir(path)  #得到文件夹下的所有文件名称
#print(files)  #这是一个列表
for file in files:
    if '政府工作报告' in file:
        f=open(path+str(file),encoding='utf-8')
        try:
            text=f.read()   #个别文件存在乱码 ，所以跳过去
        except:
            continue
        text=text.replace('&nbsp;',' ')
        text=text.replace('<BR>','\n  ')
        text=text.replace('<br>','')
        text=text.replace('</p>','')
        text=text.replace('<p>','')
        text=text.replace('</div>','')
        text=text.replace('</STRONG>','')
        text=text.replace('<STRONG>','')
        text=text.replace('<p align="center">','')
        fn=open(pathn+str(file),'w+',encoding='utf-8')
        fn.write(text)
        f.close()
        os.remove(path+str(file))  #修改一个，删除一个，这样留下来的文件就好弄了
        fn.close()

In [ ]:
#部分网页源代码和别人不一样，只识别出标题，需要改造重爬
from urllib import request, parse
from urllib.request import urlopen
from bs4 import BeautifulSoup 
import time  
import re
import os    
import sys
import pickle
import  codecs
path='D:/python/baogao_county/lianhehuinew/'
files= os.listdir(path)  #得到文件夹下的所有文件名称

head={}  #字典结构
head['User-Agent']='Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36'

for file in files:
    f=open(path+file,'r',encoding='utf-8')
    try:
        s=f.read()
    except:
        continue
    if len(s)<500:            #小于500字的全都重爬一遍。
        filen=file.replace(' ','')
        a=filen.find('年')
        i=filen[1:a-5]
        url='http://www.ssfcn.com/detailed_gh.asp?id='+str(i)+'&sid='+'\d+'
        print(url)
        req=request.Request(url,headers=head)
        resp=request.urlopen(url)
        html=resp.read()
        string=html.decode(encoding='utf-8',errors='ignore')  
        title=re.findall('<h1 style=" font-size:14px; color:#990000; padding-top:5px">(.*?)</h1>',string,re.S)
        print(title)
        text=re.findall('<div style="padding-top:16px">(.*?) <div style="width:980px',string,re.S)
       #        print(text)
        contents=title+text                   #这是个列表，每个元素是一串文字，而不是一个
#        for content in contents:
#            print(content+'\n')
        path2='D:/python/baogao_county/lianhehui2/'
        fn=open(path2+'_'+str(i)+'_'+title[0]+'.txt','w+',encoding='utf-8')  #不加这个encoding，会出错，真是累死人
        for content in contents:
            fn.write(content+'\n')
        fn.close()
        time.sleep(1.5)
        f.close()
        olddir=os.path.join(path,file)
        os.remove(olddir) 
    else:
        continue


In [ ]:
#【实际应用案例】：政府工作报告的爬取,礼拜五秘书网
#先计算一个工作报告的id
from urllib import request, parse
from urllib.request import urlopen
from urllib.error import HTTPError,URLError
from bs4 import BeautifulSoup 
import time  
import re
import requests
head={}  #字典结构
head['User-Agent']='Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36'
f=open(path+'id.txt','w+',encoding='utf-8') 
for i in range(1,208):   #到2018年4月22日，一共207页。
    url='http://www.libaiwu.com/tag/zfgzbgs/page/'+str(i)+'/'
# https://www.libaiwu.com/tag/zfgzbgs/page/2/
#    print(url)
    req=requests.get(url,headers=head)
    if req.status_code==200:
        resp=request.urlopen(url)
        html=resp.read()
        string=html.decode(encoding='utf-8',errors='ignore')  
#    print(string)
        ids=re.findall(' href="https://www.libaiwu.com/(\d+).htm',string) 
#        print(ids)
#  href="https://www.libaiwu.com/3235.htm">
        path='D:/python/baogao_county/libaiwu/'    
        for id in ids:
            f.write(id+'\n')
    else:
        continue
    time.sleep(0.5)    #爬取间隔数据
f.close()   

In [ ]:
#【实际应用案例】：政府工作报告的爬取,礼拜五秘书网
#正式爬取
from urllib import request, parse
from urllib.request import urlopen
from urllib.error import HTTPError,URLError
from bs4 import BeautifulSoup 
import time  
import re
import requests
head={}  #字典结构
head['User-Agent']='Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36'
#读取id
path='D:/python/baogao_county/libaiwu/' 
ids=open('D:/python/baogao_county/libaiwu/id.txt','r')
iss=ids.read()
iss=iss.split('\n')
print(iss[1:10])
for id in iss: 
    url='http://www.libaiwu.com/'+id+'.htm'
#        https://www.libaiwu.com/2575.htm       #模仿此公式计算而来，谨慎：部分报告有多页！！！数量较少，没有考虑
    print(url)
    req=requests.get(url,headers=head)
    if req.status_code==200:
        resp=request.urlopen(url)
        html=resp.read()
        string=html.decode(encoding='utf-8',errors='ignore')  
#    print(string)
        title=re.findall('<title>(.*?) - 礼拜五秘书网</title>',string)  
#  <title>沈阳市2013年政府工作报告 - 礼拜五秘书网</title>
        print(title)
        if '政府工作报告' in title[0]:
            stringn=re.sub('<table class="wenxue" width="100%">(.*?)</html>','',string,flags=re.S)
#        print(stringn)                                 #这里不删减一些的话，下一句匹配不干净
            text=re.findall('<p>(.*?)</p>',stringn,re.S)    #为什么有时候大写，有时候小写。。。
#        print(text)
            contents=title+text             #这是个列表，每个元素是一串文字，而不是一个
            path='D:/python/baogao_county/libaiwu/'
            f=open(path+'_'+str(id)+'_'+title[0]+'.txt','w+',encoding='utf-8') 
            for content in contents:
                f.write(content+'\n')
            f.close()
        else:
            continue
    else:
        continue
    time.sleep(0.5)           #爬取间隔数据

In [ ]:
#【实例操作】礼拜五秘书网的文件整理
import os    
import sys
import pickle
import re
import codecs
import string
import shutil
path='D:/python/baogao_county/libaiwu/'
pathn='D:/python/baogao_county/libaiwunew/'
files= os.listdir(path)  #得到文件夹下的所有文件名称
#print(files)  #这是一个列表
for file in files:
    if '政府工作报告' in file:
        f=open(path+str(file),encoding='utf-8')
        text=f.read()
        text=text.replace('&nbsp;',' ')
        text=text.replace('<b>','')
        text=text.replace('</b>','')
        text=text.replace('</strong>','')
        text=text.replace('<strong>','')
        text=text.replace('<a href="https://www.libaiwu.com/hwd/">','')
        text=text.replace('</a>','')  
        text=text.replace('</br >','')
        text=text.replace('<wbr />','')
        fn=open(pathn+str(file),'w+',encoding='utf-8')
        fn.write(text)
        f.close()
        fn.close()

In [ ]:
#【实例操作】报告重命名
#要替换，而不是另存，方便知道哪些没改成功【其实也可以存入另一个文件夹，然后原始的文件删除】
import os    
import sys
import pickle
import re
import codecs
import string
import shutil

path='D:/python/baogao_county/lianhehuinew/'
files= os.listdir(path)  #得到文件夹下的所有文件名称
#print(files)  #这是一个列表
for file in files:
    olddir=os.path.join(path,file)
    filename=os.path.splitext(file)[0]
    filetype=os.path.splitext(file)[1]
    filen=file.replace(' ','')
    a=filen.find('年')
    b=filen.find('政府工作报告')
  
    newdir=os.path.join(path,filename[a+1:b]+filename[a-4:a]+filetype);#新的文件路径
    try:
        os.rename(olddir,newdir)
    except: 
        os.remove(olddir)         #有不少重复的，将重复的删除，但这可能是将所有异常全部删除了？？

In [ ]:
baogao='_3238_2013年济南市历城区人民政府工作报告'
a=baogao.find('年')
b=baogao.find('人民政府工作报告')
print(a,b)
print(baogao[a+1:b]+baogao[a-4:a])


In [ ]:
#【实际应用案例】：政府工作报告的爬取,知县网
from urllib import request, parse
from urllib.request import urlopen
from urllib.error import HTTPError,URLError
from bs4 import BeautifulSoup 
import time  
import re
import requests
head={}  #字典结构
head['User-Agent']='Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36'
#读取id
path='D:/python/baogao_county/zhixian/' 
for id in range(9888,10110): 
    url='http://www.ahmhxc.com/gongzuobaogao/'+str(id)+'.html'
#        http://www.ahmhxc.com/gongzuobaogao/10101.html       #模仿此公式计算而来
    print(url)
    req=requests.get(url,headers=head)
    if req.status_code==200:
        resp=request.urlopen(url)
        html=resp.read()
        string=html.decode(encoding='gb2312',errors='ignore')  
#        print(string)
        title=re.findall('<div class="title"><h1>(.*?)</h1></div>',string) 
#                  <div class="title"><h1>（河南省）2018年柘城县人民政府工作报告（全文）</h1></div>
#        title=re.findall('<title>(.*?) _知县网---最有深度、最有态度、最接地气的县域大数据门户</title>',string)  
#    <title>（河南省）2018年柘城县人民政府工作报告（全文）_知县网---最有深度、最有态度、最接地气的县域大数据门户</title>
        print(title)
        text0=re.findall(' <script src="/d/js/acmsd/thea8.js"></script>(.*?)<div class="pagebreak">',string,re.S)    
#                         <p style="text-align: justify;"><span style="font-size:14px;">
#           print(text)
        contents=title+text0            #这是个列表，每个元素是一串文字，而不是一个
        for p in range(2,10):
            url='http://www.ahmhxc.com/gongzuobaogao/'+str(id)+'_'+str(p)+'.html'
#                http://www.ahmhxc.com/gongzuobaogao/10064_3.html       #模仿此公式计算而来
            req=requests.get(url,headers=head)
            if req.status_code==200:
                resp=request.urlopen(url)
                html=resp.read()
                string=html.decode(encoding='gb2312',errors='ignore') 
                text=re.findall(' <script src="/d/js/acmsd/thea8.js"></script>(.*?)<div class="pagebreak">',string,re.S) 
                contents=contents+text
            else:
                continue
        f=open(path+'_'+str(id)+'_'+title[0]+'.txt','w+',encoding='gb2312') 
        for content in contents:

            f.write(content+'\n')
        f.close()
    else:
        continue
    time.sleep(0.5)           #爬取间隔数据

In [ ]:
#【实例操作】知县网的文件整理
import os    
import sys
import pickle
import re
import codecs
import string
import shutil
path='D:/python/baogao_county/zhixian/'
pathn='D:/python/baogao_county/zhixiannew/'
files= os.listdir(path)  #得到文件夹下的所有文件名称
#print(files)  #这是一个列表
for file in files:
    if '政府工作报告' in file:
        f=open(path+str(file),encoding='gb2312')
        text=f.read()
        text=text.replace('<!--/广告位-->','')
        text=text.replace('<div>',"")
        text=text.replace('</div>','')            
        text=text.replace('&rdquo;','')
        text=text.replace('&nbsp;','')
        text=text.replace('&ldquo;','')
        text=text.replace('</td>','')
        text=text.replace('</tr>','')
        text=text.replace('</table>','')
        text=text.replace('&mdash;','')
        text=text.replace('<br />','')
        text=text.replace('</strong>','')
        text=text.replace('<strong>','')
        text=text.replace('<div class="TRS_Editor">','')
        text=text.replace('<p align="center" style="MARGIN-BOTTOM: 0px; MARGIN-TOP: 0px">','')
        text=text.replace('</p>','')
        text=text.replace('<p>','')
        text=text.replace('<!--EpointContent-->','')
        text=text.replace('<p style="TEXT-ALIGN: center">','')
        text=text.replace('</span>','')
        text=text.replace('<p style="text-align: justify;"><span style="font-size:14px;">','')
        text=text.replace('<div style="text-align: justify;">','')
        text=text.replace('<span style="font-size:14px;">','')
        text=text.replace('<p style="text-align: justify;">','')
        fn=open(pathn+str(file),'w+',encoding='utf-8')
        fn.write(text)
        f.close()
        fn.close()

In [ ]:
#【实例操作】知县网报告的重命名
#要替换，而不是另存，方便知道哪些没改成功
import os    
import sys
import pickle
import re
import codecs
import string
import shutil

path='D:/python/baogao_county/zhixiannew/'
files= os.listdir(path)  #得到文件夹下的所有文件名称
#print(files)  #这是一个列表
i=0
for file in files:
    i=i+1
    olddir=os.path.join(path,file)
    filename=os.path.splitext(file)[0]
    filetype=os.path.splitext(file)[1]
    a=file.find('年')
    b=file.find('人民政府工作报告')
  
    newdir=os.path.join(path,filename[a+1:b]+filename[a-4:a]+filetype);#新的文件路径
    try:
        os.rename(olddir,newdir)
    except: 
        os.remove(olddir)   #有不少重复的，将重复的删除，但这可能是将所有异常全部删除了？？

In [ ]:
#【实例操作】报告重命名
#思路，分别读取三份省级、地市级以及县级的代码，然后在上述文件夹当中进行匹配
import os    
import sys
import pickle
import re
import codecs
import string
import shutil
path1='d:/python/baogao_county/省份名称.txt'
path2='d:/python/baogao_county/城市名称.txt'
path3='d:/python/baogao_county/县名称1.txt'
path4='d:/python/baogao_county/县名称2.txt'
path5='d:/python/baogao_county/县名称3.txt'
prov=open(path1,'r',encoding='gb2312')
prov_name=prov.read().split('\n')
city=open(path2,'r',encoding='gb2312')
city_name=city.read().split('\n')

county1=open(path3,'r')
county_name1=county1.read().split('\n')
county2=open(path4,'r')
county_name2=county2.read().split('\n')
county3=open(path5,'r')
county_name3=county3.read().split('\n')
#print(county_name3[0:10],county_name2[0:10])

path='D:/python/baogao_county/rename/'
files= os.listdir(path)  #得到文件夹下的所有文件名称
#print(files)  #这是一个列表
for file in files:
    olddir=os.path.join(path,file)
    filename=os.path.splitext(file)[0]
    city=filename[0:-4]
    year=filename[-4:]
    filetype=os.path.splitext(file)[1]
    try:
        i=county_name3.index(city)
        cityn=county_name2[i]
        newdir=os.path.join(path,cityn+year+filetype);#新的文件路径
        os.rename(olddir,newdir)
        print(cityn)
    except:
        continue

In [ ]:
#【实际应用】国家哲学社会科学期刊数据库信息爬虫
#首先要爬取每篇没篇文章的一个网页id
from urllib import request, parse
from urllib.request import urlopen
from urllib.error import HTTPError,URLError
from bs4 import BeautifulSoup 
import time  
import re
import requests
import csv
path='D:/python/学术期刊/'   #期次文件所在目录
path2='D:/python/学术期刊/id/'  # 论文id保存文件夹目录
f=open(path2+'金融研究.csv','a',encoding='utf-8',newline='')  #需要手工修改期刊名称
journal_id='97926X'         #需要手工修改期刊id，期刊id为手工整理
c=csv.writer(f)
issue=open(path+'期次.csv','r',encoding='utf-8')
issueid=issue.read()
issueids=issueid.split('\n')
issueids=issueids[0:205]        #多出一行，末尾为空，搞不清楚原因，可能是上一行\n切割导致的,第一个也有问题
head={}  #字典结构
head['User-Agent']='Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Mobile Safari/537.36'
for i in range(1,217):   #到2001年1-2017年12月，204个月。
    url=' http://www.nssd.org/journal/cn/'+journal_id+'/'+str(issueids[i])+'/'
    print(i,url)
    req=requests.get(url,headers=head)
    if req.status_code==200:
        resp=request.urlopen(url)
        html=resp.read()
        string=html.decode(encoding='utf-8',errors='ignore')  
        ids=re.findall('article_detail.aspx\?id=(.*?)">(.*?)</a></td>',string,re.S)   #?id当中的？需要转义符处理一下        
        for id in ids:
            c.writerow((id[0],str(issueids[i])))
    else:
        continue
f.close()

In [ ]:
#【实际应用】国家哲学社会科学期刊数据库信息爬虫
#根据上一篇爬取的网页，再进行正式爬取作者及单位信息，其他信息不要
from urllib import request, parse
from urllib.request import urlopen
from urllib.error import HTTPError,URLError
from bs4 import BeautifulSoup 
import time  
import re
import requests
import csv
path1='D:/python/学术期刊/id_new/'  #论文id读取目录
path2='D:/python/学术期刊/paper/'   #论文信息保存目录
f=open(path1+'经济研究.txt','r',encoding='utf-8')    #需要手工修改期刊名称
fn=open(path2+'经济研究paper2.csv','w+',encoding='utf-8',newline='')   #需要手工修改期刊
c=csv.writer(fn)
c.writerow(('paper_id','title','journal','authors','units'))
head={}  #字典结构
head['User-Agent']='Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Mobile Safari/537.36'

ids=f.read().split()
for id in ids:
    if id.isdigit():
        url=' http://www.nssd.org/articles/article_detail.aspx?id='+str(id)
#   http://www.nssd.org/articles/article_detail.aspx?id=4365341
    else:
        continue
    print(url)
    req=requests.get(url,headers=head)
    if req.status_code==200:
        resp=request.urlopen(url)
        html=resp.read()
        string=html.decode(encoding='utf-8',errors='ignore')  
        #string= html.decode('utf-8').encode(type)
        #print(string)
        title=re.findall('<title>(.*?)-国家哲学社会科学学术期刊数据库</title>',string,re.S) 
        if title==[]:
            title=['no_title']
        journal=re.findall('.*【期&#12288;&#12288;刊】.*title=\'\'>(.*?)</a>  <i>.*',string,re.S)
        if journal==[]:
            journal=['no_journal']
        authors0=re.findall('【作&#12288;&#12288;者】(.*?)【作者单位】',string,re.S)
        if authors0==[]:
            authors=['no_author']
        else:
            authors=re.findall('title=\'\'>(.*?)</a>',authors0[0],re.S)
        if authors==[]:
            authors=['no_author']
        units0=re.findall('【作者单位】(.*?)【期&#12288;&#12288;刊】',string,re.S)
        if units0==[]:
            units=['no_unit']
        else:
            units=re.findall('title=\'\'>(.*?)</a>',units0[0],re.S)
        if units==[]:
            units=['no_unit']
        print(title)
        print(journal)
        print(authors)
        print(units)
        c.writerow((id,title[0],journal[0],";".join(authors),";".join(units)))
    else:
        continue
f.close() 
fn.close()

In [ ]:
#中国知网数据爬虫
# -*-coding:utf-8-*-
import pymysql
from bs4 import BeautifulSoup
import random
import time
import requests
import random

class Cnki:
    agents = [
        "Mozilla/5.0 (Linux; U; Android 2.3.6; en-us; Nexus S Build/GRK39F) AppleWebKit/533.1 (KHTML, like Gecko) Version/4.0 Mobile Safari/533.1",
        "Avant Browser/1.2.789rel1 (http://www.avantbrowser.com)",
        "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/532.5 (KHTML, like Gecko) Chrome/4.0.249.0 Safari/532.5",
        "Mozilla/5.0 (Windows; U; Windows NT 5.2; en-US) AppleWebKit/532.9 (KHTML, like Gecko) Chrome/5.0.310.0 Safari/532.9",
        "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/534.7 (KHTML, like Gecko) Chrome/7.0.514.0 Safari/534.7",
        "Mozilla/5.0 (Windows; U; Windows NT 6.0; en-US) AppleWebKit/534.14 (KHTML, like Gecko) Chrome/9.0.601.0 Safari/534.14",
        "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.14 (KHTML, like Gecko) Chrome/10.0.601.0 Safari/534.14",
        "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.20 (KHTML, like Gecko) Chrome/11.0.672.2 Safari/534.20",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/534.27 (KHTML, like Gecko) Chrome/12.0.712.0 Safari/534.27",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/13.0.782.24 Safari/535.1",
        "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/535.2 (KHTML, like Gecko) Chrome/15.0.874.120 Safari/535.2",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.7 (KHTML, like Gecko) Chrome/16.0.912.36 Safari/535.7",
        "Mozilla/5.0 (Windows; U; Windows NT 6.0 x64; en-US; rv:1.9pre) Gecko/2008072421 Minefield/3.0.2pre",
    ]
    cookie = {
         "Cookie": 'Ecp_notFirstLogin=zXyitT; Ecp_ClientId=3171111123202637951; cnkiUserKey=56ef38ce-f0bf-6489-1c03-c504b7797c78; _pk_id=2a7db67d-0ddf-49a7-a737-3fbcf821bd4c.1523005065.28.1530758366.1530758366.; RsPerPage=50; __uid=722d21de2d1490a507013e4d2fed2fca; amid=1f66a89a-01ae-4ba2-b293-7adc79758a44; UM_distinctid=16712a3629191-00a248c7d224f-3c604504-e1000-16712a3629210e; ASP.NET_SessionId=2dgglsciss52obcr1rgoxrog; SID_kns=123117; SID_klogin=125144; SID_kinfo=125105; KNS_SortType=; SID_krsnew=125132; SID_kcms=124120; SID_knsdelivery=125122; Ecp_lout=1; IsLogin=; CNZZDATA3258975=cnzz_eid%3D1492256385-1517302996-http%253A%252F%252Fnvsm.cnki.net%252F%26ntime%3D1542973532; _pk_ref=%5B%22%22%2C%22%22%2C1542977641%2C%22http%3A%2F%2Fwww.cnki.net%2F%22%5D; _pk_ses=*; LID=WEEvREcwSlJHSldTTEYzVnB3ZDByTWVaZzIycTFDVS8rQ0YvVFlZZ29jcz0=$9A4hF_YAuvQ5obgVAqNKPCYcEjKensW4IQMovwHtwkF4VYPoHbKxJw!!; Ecp_session=1; Ecp_LoginStuts=%7B%22IsAutoLogin%22%3Afalse%2C%22UserName%22%3A%22guofengsfi2%22%2C%22ShowName%22%3A%22guofengsfi2%22%2C%22UserType%22%3A%22jf%22%2C%22r%22%3A%22zXyitT%22%7D; c_m_LinID=LinID=WEEvREcwSlJHSldTTEYzVnB3ZDByTWVaZzIycTFDVS8rQ0YvVFlZZ29jcz0=$9A4hF_YAuvQ5obgVAqNKPCYcEjKensW4IQMovwHtwkF4VYPoHbKxJw!!&ot=11/25/2018 21:14:10; c_m_expire=2018-11-25 21:14:10',
        }  # 【这里隔段时间需要自己填写cookies】
    UA = random.choice(agents)
    header = {'User-Agent': UA}
    cnx = pymysql.connect(user='root', password='guo840306', host='localhost', database='cnki6',charset='utf8')  #填写数据库信息
    cur = cnx.cursor()

    def paper_info(self):
        for i in range(1,14):  #【页数以及下面的网页需要自己填写】
            time.sleep(random.choice([1,2,3]))
            url = 'http://kns.cnki.net/kns/brief/brief.aspx?curpage={}&RecordsPerPage=50&QueryID=11&ID=&turnpage=1&tpagemode=L&dbPrefix=CJFQ&Fields=&DisplayMode=listmode&PageName=ASP.brief_result_aspx&isinEn=1'.format(i)
            print(url)
            req = requests.get(url, headers=self.header, cookies = self.cookie)
            soup1 = BeautifulSoup(req.text, 'lxml').find('table', class_='GridTableContent').find_all('tr', bgcolor = '#ffffff')
            soup2 = BeautifulSoup(req.text, 'lxml').find('table', class_='GridTableContent').find_all('tr', bgcolor = '#f6f7fb')
            for s1 in soup1:
                paper_name = ''
                author = ''
                author_unit = ''
                mag_name = ''
                year_period = ''
                cited = 0
                download = 0
                gjc = ''
                zhaiyao = ''
                fund = ''
                class_num = ''
                page_num = 0
                page_range = ''

                td = s1.find_all('td')
                paper_name =td[1].get_text().strip('\n').strip()
                author = td[2].get_text().strip('\n').strip()
                mag_name = td[3].get_text().strip('\n').strip()
                year_period = td[4].get_text().strip('\n').strip()
                cited = td[5].get_text().strip('\n').strip()
                if cited ==  '':
                    cited = 0
                download = td[6].get_text().strip('\n').strip()
                if download == '':
                    download = 0

                href = 'http://kns.cnki.net/KCMS' + td[1].find('a')['href'].replace('/kns', '').replace(' ','')
                req2 = requests.get(href, headers=self.header, cookies = self.cookie)
                time.sleep(0.5)
                soup3 = BeautifulSoup(req2.text, 'lxml').find('div', class_='wxBaseinfo')

                if soup3.find('span', id='ChDivSummary') != None:
                    zhaiyao = soup3.find('span', id='ChDivSummary').get_text()

                p = soup3.find_all('p')
                for p1 in p:
                    if p1.find('label', id='catalog_KEYWORD') != None:
                        gjc = p1.get_text().replace('\n', '').replace('                  ','').strip()
                        print(gjc)
                    if p1.find('label', id='catalog_FUND') != None:
                        fund = p1.get_text().replace('\n', '').replace('                  ', '').strip()
                        print(fund)
                    if p1.find('label', id='catalog_ZTCLS') != None:
                        class_num = p1.get_text().replace('\n', '').strip()
                        print(class_num)

                if BeautifulSoup(req2.text, 'lxml').find('div', class_='orgn')!= None:
                    author_unit_list = BeautifulSoup(req2.text, 'lxml').find('div', class_='orgn').find_all('span')
                    for s in author_unit_list:
                        author_unit = author_unit + '  ' + s.get_text()

                if BeautifulSoup(req2.text, 'lxml').find('div', class_='dllink-down')!= None:
                    page_list = BeautifulSoup(req2.text, 'lxml').find('div', class_='dllink-down').find('div', class_='total').find_all('span')
                    for p in page_list:
                        if p.find('label').get_text() == '页码：':
                            page_range = p.find('b').get_text()
                        if p.find('label').get_text() == '页数：':
                            page_num = p.find('b').get_text()

                print(paper_name + '  ' + author + '    ' + mag_name + '    ' + year_period)

                sql = 'INSERT INTO cnki  (`paper_name`,`author`,`author_unit`, `mag_name`, `year_period` ,`cited`,\
                  `download`, `gjc`,  `zhaiyao`, `fund`, `class_num`, `page_num`, `page_range`) \
                    VALUES (%(paper_name)s, %(author)s, %(author_unit)s, %(mag_name)s, %(year_period)s,\
                      %(cited)s, %(download)s, %(gjc)s, %(zhaiyao)s, %(fund)s, %(class_num)s, %(page_num)s, %(page_range)s)'
                value = {
                    'paper_name':paper_name,
                    'author':author,
                    'author_unit':author_unit,
                    'mag_name':mag_name,
                    'year_period':year_period,
                    'cited':cited,
                    'download':download,
                    'gjc':gjc,
                    'zhaiyao':zhaiyao,
                    'fund':fund,
                    'class_num':class_num,
                    'page_num':page_num,
                    'page_range':page_range
                }
                self.cur.execute(sql, value)
                self.cnx.commit()

            for s2 in soup2:
                paper_name = ''
                author = ''
                author_unit = ''
                mag_name = ''
                year_period = ''
                cited = 0
                download = 0
                gjc = ''
                zhaiyao = ''
                fund = ''
                class_num = ''
                page_num = 0
                page_range = ''

                td = s2.find_all('td')
                paper_name =td[1].get_text().strip('\n').strip()
                author = td[2].get_text().strip('\n').strip()
                mag_name = td[3].get_text().strip('\n').strip()
                year_period = td[4].get_text().strip('\n').strip()
                cited = td[5].get_text().strip('\n').strip()
                if cited ==  '':
                    cited = 0
                download = td[6].get_text().strip('\n').strip()
                if download == '':
                    download = 0
                href1 = 'http://kns.cnki.net/KCMS' + td[1].find('a')['href'].replace('/kns', '').replace(' ','')

                req3 = requests.get(href1, headers=self.header, cookies = self.cookie)
                time.sleep(0.5)
                soup4 = BeautifulSoup(req3.text, 'lxml').find('div', class_='wxBaseinfo')

                if soup4.find('span', id = 'ChDivSummary') != None:
                    zhaiyao = soup4.find('span', id = 'ChDivSummary').get_text()

                pp = soup4.find_all('p')
                for p2 in pp:
                    if p2.find('label', id='catalog_KEYWORD') != None:
                        gjc = p2.get_text().replace('\n', '').replace('                  ', '').strip()
                        print(gjc)
                    if p2.find('label', id='catalog_FUND') != None:
                        fund = p2.get_text().replace('\n', '').replace('                  ', '').strip()
                        print(fund)
                    if p2.find('label', id='catalog_ZTCLS') != None:
                        class_num = p2.get_text().replace('\n', '').strip()
                        print(class_num)

                if BeautifulSoup(req3.text, 'lxml').find('div', class_='orgn') != None:
                    author_unit_list = BeautifulSoup(req3.text, 'lxml').find('div', class_='orgn').find_all('span')
                    for s in author_unit_list:
                        author_unit = author_unit + '  ' + s.get_text()

                if BeautifulSoup(req3.text, 'lxml').find('div', class_='dllink-down') != None:
                    page_list2 = BeautifulSoup(req3.text, 'lxml').find('div', class_='dllink-down').find('div',class_='total').find_all('span')
                    for p in page_list2:
                        if p.find('label').get_text() == '页码：':
                            page_range = p.find('b').get_text()
                        if p.find('label').get_text() == '页数：':
                            page_num = p.find('b').get_text()
                print(paper_name + '  ' + author + '    ' + mag_name + '    ' + year_period)

                sql = 'INSERT INTO cnki (`paper_name`,`author`,`author_unit`, `mag_name`, `year_period` ,`cited`,\
                      `download`, `gjc`, `zhaiyao`, `fund`, `class_num`, `page_num`, `page_range`) \
                       VALUES (%(paper_name)s, %(author)s, %(author_unit)s, %(mag_name)s, %(year_period)s,\
                   %(cited)s, %(download)s, %(gjc)s, %(zhaiyao)s, %(fund)s, %(class_num)s, %(page_num)s, %(page_range)s)'
                value = {
                    'paper_name': paper_name,
                    'author': author,
                    'author_unit': author_unit,
                    'mag_name': mag_name,
                    'year_period': year_period,
                    'cited': cited,
                    'download': download,
                    'gjc': gjc,
                    'zhaiyao': zhaiyao,
                    'fund': fund,
                    'class_num': class_num,
                    'page_num': page_num,
                    'page_range': page_range
                }
                self.cur.execute(sql, value)
                self.cnx.commit()


if __name__ == '__main__':
    cnki = Cnki()  # 调用Cnki类，创建实例
    cnki.paper_info()
    print("**************爬取结束******************")

In [ ]:
#中国哲学社会期刊数据库网站爬取
import pymysql
from bs4 import BeautifulSoup
import random
import time
import requests

class Nssd:

    agents = [
        "Mozilla/5.0 (Linux; U; Android 2.3.6; en-us; Nexus S Build/GRK39F) AppleWebKit/533.1 (KHTML, like Gecko) Version/4.0 Mobile Safari/533.1",
        "Avant Browser/1.2.789rel1 (http://www.avantbrowser.com)",
        "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/532.5 (KHTML, like Gecko) Chrome/4.0.249.0 Safari/532.5",
        "Mozilla/5.0 (Windows; U; Windows NT 5.2; en-US) AppleWebKit/532.9 (KHTML, like Gecko) Chrome/5.0.310.0 Safari/532.9",
        "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/534.7 (KHTML, like Gecko) Chrome/7.0.514.0 Safari/534.7",
        "Mozilla/5.0 (Windows; U; Windows NT 6.0; en-US) AppleWebKit/534.14 (KHTML, like Gecko) Chrome/9.0.601.0 Safari/534.14",
        "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.14 (KHTML, like Gecko) Chrome/10.0.601.0 Safari/534.14",
        "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.20 (KHTML, like Gecko) Chrome/11.0.672.2 Safari/534.20",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/534.27 (KHTML, like Gecko) Chrome/12.0.712.0 Safari/534.27",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/13.0.782.24 Safari/535.1",
        "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/535.2 (KHTML, like Gecko) Chrome/15.0.874.120 Safari/535.2",
    ]

    cnx = pymysql.connect(user='root', password='guo840306', host='localhost', database='nssd',charset='utf8mb4')
    cur = cnx.cursor()


    def get_info(self, journal):

        UA = random.choice(self.agents)
        header = {'User-Agent': UA}

        for year in range(2001, 2018):
            url = 'http://www.nssd.org/journal/cn/%sB/%s01/?lis=1' % (str(journal), str(year))
            print(url)
            response = requests.get(url, headers = header)
            time.sleep(0.5)
            #print(response.text)
            soup = BeautifulSoup(response.text, 'lxml')
            li_list = soup.find('div', id='numlist').find_all('li')
            for li in range(0,len(li_list)):  # 期数len(li_list)为期总数
                href = 'http://www.nssd.org/' + li_list[li].find('a')['href']
                response2 = requests.get(href, headers = header)
                time.sleep(0.5)
                soup2 = BeautifulSoup(response2.text, 'lxml')
                tr_list = soup2.find('table', class_='t_list').find_all('tr')
                for tr in tr_list:
                    if tr.find('td'):
                        paper_name = ''
                        author = ''
                        author_unit = ''
                        mag_name = ''
                        detail_url =  'http://www.nssd.org/' +  tr.find('td').find('a')['href']
                        response3 = requests.get(detail_url, headers = header)
                        time.sleep(0.5)
                        soup3 = BeautifulSoup(response3.text, 'lxml')
                        detail = soup3.find('div', class_='summary')
                        paper_name = detail.find('h1').get_text()
                        print(paper_name)
                        p_list = detail.find('div', class_='article_detail').find_all('p')
                        for p in p_list:
                            if p.find('strong').get_text() == '【作　　者】':
                                a_list = p.find_all('a')
                                for a in a_list:
                                    author = author + a.get_text() + '  '
                            if p.find('strong').get_text() == '【作者单位】':
                                a_list = p.find_all('a')
                                for a in a_list:
                                    author_unit = author_unit + a.get_text() + '  '
                            if p.find('strong').get_text() == '【期　　刊】':
                                mag_name = p.get_text().split(' ')[0] + ' ' + p.get_text().split(' ')[-1]
                                mag_name = mag_name.replace('【期　　刊】', '')
                        print(author_unit)
                        print(author)
                        print(mag_name)

                        sql = 'INSERT INTO nssd (`paper_name`, `author`, `author_unit`,  `mag_name`) VALUES ( %(paper_name)s, %(author)s, %(author_unit)s, %(mag_name)s)'
                        value = {
                            'paper_name': paper_name,
                            'author': author,
                            'author_unit': author_unit,
                            'mag_name': mag_name
                        }
                        self.cur.execute(sql, value)
                        self.cnx.commit()

if __name__ == '__main__':
    nssd = Nssd()
    journal = [98338]           #期刊代码
    for jour in journal:
        nssd.get_info(jour)
